In [1]:
from notion_client import Client
from canvasapi import Canvas, assignment
import canvasapi as cnv
from tokens import *
from datetime import datetime,timedelta
import pytz

In [2]:
nclient = Client(auth=notion_token)
API_KEY = canvas_token
canvas = Canvas(API_URL,API_KEY)
cnvclient = canvas.get_current_user()
courses = []
enrollment_term_id = 164
for course in cnvclient.get_courses():
    if course.__dict__['enrollment_term_id'] == enrollment_term_id:
        courses.append(course)
subjects = {}

In [16]:
marker = nclient.databases.query(**{"database_id":db_id,"filter":{"property":"Name","title":{"equals":"Marker"}}})['results'][0]

In [9]:
marker[0]["properties"]

{'Due Date': {'id': '%3CL%5B%60', 'type': 'date', 'date': None},
 'Scheduled Date': {'id': 'F%3Dd%3A', 'type': 'date', 'date': None},
 'Link': {'id': 'F%7CBr', 'type': 'url', 'url': None},
 'Blocked by': {'id': 'I%7COD',
  'type': 'relation',
  'relation': [],
  'has_more': False},
 'Project': {'id': 'N%7DUj',
  'type': 'relation',
  'relation': [],
  'has_more': False},
 'Subject': {'id': '%60QUQ', 'type': 'select', 'select': None},
 'Time to complete': {'id': 'i%3D%7Bc',
  'type': 'select',
  'select': {'id': 'b261419e-254f-44b9-8788-1532a54646fb',
   'name': 'About an hour',
   'color': 'red'}},
 'Blocking': {'id': 'p%3FZ%3D',
  'type': 'relation',
  'relation': [],
  'has_more': False},
 'Completion Progress': {'id': 'pC%7D%3E', 'type': 'number', 'number': None},
 'Status': {'id': 'yVNV',
  'type': 'status',
  'status': {'id': '2', 'name': 'Week Backlog', 'color': 'yellow'}},
 'Name': {'id': 'title',
  'type': 'title',
  'title': [{'type': 'text',
    'text': {'content': 'Marker', 

In [10]:
def get_schema(assignment:assignment,subject_dict:dict,course:str):
    if course in subject_dict.keys():
        subject = subject_dict[course]
    else:
        subject = {'name':course}
    schema = marker["properties"]
    schema.update({
    'Name': {"title": [{"type":"text","text":{"content":assignment.name}}]},
    "Subject":{"select":subject},
    "Due Date":{"date":{"start":datetime.strptime(assignment.due_at,"%Y-%m-%dT%H:%M:%S%z").astimezone(pytz.timezone('US/Eastern')).strftime("%Y-%m-%dT%H:%M:%S%z")}},
    "Link":{"url":assignment.html_url},
    })
    return schema

In [11]:
def add_assignment(assignment:assignment,subject_dict:dict,course:str):
    return nclient.pages.create(**{"parent":{"type":"database_id","database_id":db_id},"properties":get_schema(assignment,subjects,course)})


In [12]:
def update_course(course:cnv.course.Course,subject_dict:dict):
    course_assignments = [y for y in cnvclient.get_assignments(course)]
    for x in course_assignments:
        if course.name in subject_dict.keys():
            subject = subject_dict[course.name]['name']
        else:
            subject = course.name
        if len(nclient.databases.query(**{"database_id":db_id,"filter":{"and":[{"property":"Name","title":{"contains":x.name}},{"property":"Subject","select":{"equals":subject}}]}})['results'])<1 and '''x.get_submission(kiran).submitted_at==None''' and x.due_at!=None:
            print(x)
            print(add_assignment(assignment=x,subject_dict=subject_dict,course=course.name)['properties']['Name'])

In [ ]:
def update_all(courses:list,subject_dict:dict):
    backlog = "Backlog"
    week_backlog = "Week Backlog"
    backloglist = nclient.databases.query(**{"database_id":db_id,"filter":{"property":"Status","status":{"equals":"Backlog"}}})['results']
    for page in backloglist:
        date = datetime.fromisoformat(page['properties']['Due Date']['date']['start'])
        if date.date()-datetime.today().date()<timedelta(days=7):
            nclient.pages.update(page['id'],**{'properties':{'Status':{'status':{'name':week_backlog}}},"sorts":[{"property":"Due Date","direction":"ascending"}]})
    for x in courses:
        update_course(x,subject_dict)

In [ ]:
nclient.databases.__module__

In [ ]:
backloglist = nclient.databases.query(**{"database_id":db_id,"filter":{"property":"Status","status":{"equals":"Backlog"}},"sorts":[{"property":"Due Date","direction":"ascending"}]})['results']

In [ ]:
weekbackloglist = nclient.databases.query(**{"database_id":db_id,"filter":{"property":"Status","status":{"equals":"Week Backlog"}}})['results']

In [ ]:
nclient.databases.query(**{"database_id":db_id,"filter":{"property":"Name","title":{"contains":"PD1 - Problem Definition: User Persona & Pains"}}})['results'][0]['properties']['Status']['status'] == backloglist[0]['properties']['Status']['status']

In [ ]:
for x in weekbackloglist:
    print(x['properties']['Name']['title'][0]['plain_text'])

In [ ]:
print(len(backloglist))

In [ ]:
for x in backloglist:
    print(x['properties']['Name']['title'][0]['plain_text'])

In [ ]:
update_all(courses=courses,subject_dict=subjects)

In [ ]:
backloglist[1]

In [ ]:
def add_links(course:cnv.course.Course):
    course_assignments = [y for y in kiran.get_assignments(course)]
    for x in course_assignments:
        if x.get_submission(kiran).submitted_at == None and x.due_at!=None:
            print(x)
            page = nclient.databases.query(**{"database_id":db_id,"filter":{"property":"Name","title":{"contains":x.name}}})['results'][0]
            nclient.pages.update(page['id'],**{'properties':{'Link':{'url':x.html_url}}})

In [ ]:
def update_times(course:cnv.course.Course):
    course_assignments = [y for y in kiran.get_assignments(course)]
    for x in course_assignments:
        if x.get_submission(kiran).submitted_at == None and x.due_at!=None:
            print(x)
            page = nclient.databases.query(**{"database_id":db_id,"filter":{"and":[{"property":"Name","title":{"contains":x.name}},{"property":"Subject","select":{"equals":subjects[course.name]['name']}}]}})['results'][0]
            nclient.pages.update(page['id'],**{'properties':{'Due Date':{"date":{"start":datetime.strptime(x.due_at,"%Y-%m-%dT%H:%M:%S%z").astimezone(pytz.timezone('US/Eastern')).strftime("%Y-%m-%dT%H:%M:%S%z")}}}})

In [ ]:
def find_duplicates(course_list):
    for c in course_list:
        course_assignments = [y for y in kiran.get_assignments(c)]
        for x in course_assignments:
            subject = c.name
            if len(nclient.databases.query(**{"database_id":db_id,"filter":{"and":[{"property":"Name","title":{"contains":x.name}},{"property":"Subject","select":{"equals":subject}}]}})['results'])>0 and '''x.get_submission(kiran).submitted_at==None''' and x.due_at!=None:
                print(x)

In [ ]:
def remove_duplicates(course_list):
    for c in course_list:
        course_assignments = [y for y in kiran.get_assignments(c)]
        for x in course_assignments:
            subject = c.name
            qresults = nclient.databases.query(**{"database_id":db_id,"filter":{"and":[{"property":"Name","title":{"contains":x.name}},{"property":"Subject","select":{"equals":subject}}]}})['results']
            if len(qresults)>1 and '''x.get_submission(kiran).submitted_at==None''' and x.due_at!=None:
                for x in range(len(qresults)-1):
                    currentpage = nclient.pages.update(qresults[x]['id'],**{'archived':True})